TITLE: Analysis of User Knowledge Modeling: Predicting an individuals intelligence level

INTRODUCTION: Understanding knowledge is key for assessing an individual's ability to learn and apply new information. Generally, this is a challenging problem due to the fact that many people are skilled in varied fields, and thus don't necessarily have a clear idea as to how they stack up among others. Using different classifications ranging from "very_low" to "high", This data set contains a variety of potential predictor values that may help us predict an individual's knowledge level. In this analysis, we'll ask the following question: "Can STG (Study Time for Goal) and PEG (Exam Performance for Goal) act as predictor values to help us figure out a user's knowledge level?" We'll use the following data set: https://archive.ics.uci.edu/static/public/257/user+knowledge+modeling.zip

In [11]:
#a few extra things need to be imported in order to remotely read the excel file within the zip file
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

#finding and reading the excel file
zip_file_url = 'https://archive.ics.uci.edu/static/public/257/user+knowledge+modeling.zip'
response = requests.get(zip_file_url)
zip_file = ZipFile(BytesIO(response.content))
excel_file = zip_file.namelist()[0]
initial_read = pd.read_excel(zip_file.open(excel_file), sheet_name='Training_Data')

#dropping some extra columns not relevant to our analysis. knowledge_training should output just the data and the classification
knowledge_training_initial = initial_read.drop(columns=['Unnamed: 6', 'Unnamed: 7', 'Attribute Information:'])
#further dropping columns that contain data, but are not to be used as predictor variables (everything but STG and PEG, study time and exam performance)
knowledge_training = knowledge_training_initial.drop(columns=['SCG', 'STR', 'LPR'])

#renaming columns to be legible -- UNS didn't change for some reason idk why
knowledge_training = knowledge_training.rename(columns={'STG': 'study_time', 'PEG': 'exam_performance', 'UNS': 'knowledge'})

knowledge_training

,study_time,exam_performance,UNS
0,0.00,0.00,very_low
1,0.08,0.90,High
2,0.06,0.33,Low
3,0.10,0.30,Middle
4,0.08,0.24,Low
...,...,...,...
253,0.61,0.58,High
254,0.78,0.60,Middle
255,0.54,0.77,High
256,0.50,0.26,Middle
